# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем все необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Загрузим датафрейм, посмотрим содержимое.

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data.head(10))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Посмотрим инфо.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков в данных нет. Проверим дубликаты

In [4]:
display('Дубликатов в таблице', data.duplicated().sum())

'Дубликатов в таблице'

0

Дубликатов тоже нет. Можем приступать к разбивке на выборки.

Выводы: 1) Загрузили библиотеки. 2) Загрузили датафрейм. 3) Проверили датафрейм, данные в порядке: нет пропусков, нет дубликатов, столбцы названы корректно.

## Разбейте данные на выборки

Выделим отдельно признаки и целевой признак. Целевой признак- тариф, ему соответствует столбец "is_ultra".

In [5]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

Теперь разобьем данные на выборки, по соотношению 3:1:1. Начнем с выделения тестовой выборки, на нее должно придтись 20% исходных данных.

In [6]:
data_features, features_test, data_target, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

Теперь выделим уже 0.25 от выборок data_features и data_target- под валидационную выборку.

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(data_features, 
                                                                              data_target, test_size=0.25, random_state=12345)

Проверим результат.

In [8]:
features_test.info()
features_train.info()
features_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1415 to 1196
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
dtypes: float64(4)
memory usage: 25.1 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2656 to 510
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
dtypes: float64(4)
memory usage: 75.3 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2699 to 1806
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minu

Данные поделены корректно. Можем приступать к исследованию моделей.

## Исследуйте модели

Начнем по порядку- с решающего дерева.

In [9]:
best_params = []
best_accuracy = 0
#создаем циклы, которые будут перебирать значения параметров.
#цикл максимальной глубины дерева
for max_depth in range(2,10):
#цикл минимального колчичества объектов в узле    
    for min_samples_split in range(2,6):
#цикл минимального количества объектов в листе        
        for min_samples_leaf in range(1,10):
            # Проверим модель с текущими параметрами
            model_classification = DecisionTreeClassifier(random_state = 123,
                                                  max_depth = max_depth,
                                                  min_samples_split = min_samples_split,
                                                  min_samples_leaf = min_samples_leaf)                                                    
            # обучение модели на тренировочной выборке
            model_classification.fit(features_train, target_train)
            # поиск предсказаний модели на валидационной выбоке
            predictions = model_classification.predict(features_valid)
            # вычисление точности модели методом accuracy_score
            accuracy = accuracy_score(target_valid, predictions)
            # если текущее значение точности выше предыдущего лучшего значения, 
            # сохраняем параметры модели и текущую точность
            if accuracy > best_accuracy:
                best_params = [max_depth, min_samples_split, min_samples_leaf]
                best_accuracy = accuracy

display('Самая высокая точность:', best_accuracy)
display('Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf', best_params)

'Самая высокая точность:'

0.7916018662519441

'Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf'

[7, 2, 8]

С первой попытки выполнили условия задачи- точность выше 0,75
Попробуем улучшить результат на других моделях.

Теперь попробуем метод случайного леса.
Гиперпараметры будут аналогичными, но добавим n_estimators- количество деревьев.

In [10]:
best_params = []
best_accuracy = 0
#создаем циклы, которые будут перебирать значения параметров.
#цикл максимальной глубины дерева
for max_depth in range(2,10):
#цикл минимального колчичества объектов в узле    
    for min_samples_split in range(2,6):
#цикл минимального количества объектов в листе        
        for min_samples_leaf in range(1,10):
#цикл количества деревьев 
            for n_estimators in (1,10):
                # Проверим модель с текущими параметрами
                model_classification = RandomForestClassifier(random_state = 123,
                                                      max_depth = max_depth,
                                                      min_samples_split = min_samples_split,
                                                      min_samples_leaf = min_samples_leaf,
                                                      n_estimators = n_estimators)                                                    
                # обучение модели на тренировочной выборке
                model_classification.fit(features_train, target_train)
                # поиск предсказаний модели на валидационной выбоке
                predictions = model_classification.predict(features_valid)
                # вычисление точности модели методом accuracy_score
                accuracy = accuracy_score(target_valid, predictions)
                # если текущее значение точности выше предыдущего лучшего значения, 
                # сохраняем параметры модели и текущую точность
                if accuracy > best_accuracy:
                    best_params = [max_depth, min_samples_split, min_samples_leaf, n_estimators]
                    best_accuracy = accuracy

display('Самая высокая точность:', best_accuracy)
display('Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf, n_estimators', best_params)

'Самая высокая точность:'

0.7931570762052877

'Лучшие значения параметров max_depth, min_samples_split, min_samples_leaf, n_estimators'

[8, 2, 8, 10]

Точность практически не отличается: 0.79

Теперь попробуем логистическую регрессию.
Используем параметр max_iter- максимум итераций, от 100 до 2000 с шагом в 100

In [11]:
# в параметрах best_params и best_accuracy будем хранить наилучшие параметры для модели и наивысшую точность соответственно
best_params = []
best_accuracy = 0

# проходим в циклах по всем выбранным гиперпараметрам
for max_iter in range(1, 10):
    max_iter = max_iter * 100
    # инициация модели с текущими гиперпараметрами
    model_classification = LogisticRegression(random_state=123, max_iter = max_iter)
    # обучение модели на тренировочной выборке
    model_classification.fit(features_train, target_train)
    # поиск предсказаний модели на валидационной выбоке
    predictions = model_classification.predict(features_valid)
    # вычисление точности модели методом accuracy_score
    accuracy = accuracy_score(target_valid, predictions)
    # если текущее значение точности выше предыдущего лучшего значения, 
    # сохраняем параметры модели и текущую точность
    if accuracy > best_accuracy:
        best_params = max_iter
        best_accuracy = accuracy
display('Самая высокая точность:', best_accuracy)
display('Лучшие значения параметров max_iter', best_params)

'Самая высокая точность:'

0.7262830482115086

'Лучшие значения параметров max_iter'

100

Точность получилась ниже, чем в прошлых методах.

**Выводы:**
    
Проверили три модели. У логистической регрессии результат получился сильно ниже, чем у двух других моделей.

Для моделей "случайный лес" и "решающее дерево" использовали три и более гиперпараметров.

Модели "случайный лес" и "решающее дерево" показали наибольшую точность. Поскольку результат у случайного леса чуть выше (0.793 против 0.791 у случайного леса, оставим именно его для проверки на тестовой выборке.

## Проверьте модель на тестовой выборке

In [12]:
model_classification = RandomForestClassifier(random_state=123,
                                      max_depth=8,
                                      min_samples_split=2,
                                      min_samples_leaf=8,
                                      n_estimators = 10)
#обучим модель на выборке большего размера- обучающей+валидационной
model_classification.fit(data_features, data_target)
predictions = model_classification.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
display('Точность модели на тестовой выборке:', accuracy)

Точность модели на тестовой выборке: 0.7978227060653188


На тестовой выборке метод случайного леса показывает результат лучше целевого.

## (бонус) Проверьте модели на адекватность

Сделаем проверку с помощью DummyClassifier, используем метод most_frequent.

In [14]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
predictions = dummy_clf.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
display(accuracy)

0.6889580093312597

Результат получается ниже, чему у любого их трех проверенных методов. Следовательно, все три метода эффективны, хоть и в разной степени.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

## Вывод

1) Изучили данные. Дубликатов не обнаружили, аномалий не обнаружили.
2) Разбили данные на выборки.
3) Проанализировали три модели: Решающее дерево, Случайный лес, Логистическая регрессия. Два из них (решающее дерево и случайный лес), дали результат выше требуемого в 0,75
4) Обнаружили на тестовой выборке, что больший результат дает случайный лес.
5) Проверили зависимость точности от гиперпараметра n_estimators для модели Случайный лес. Результат соответсвует тому, что мы наблюдали в исследовании ранее.